## Gymnasium environments

This Section shows how you can make and use the `gym` environments that interface with the simulator.

In [1]:
import os
from pathlib import Path
import torch
import mediapy

# Set working directory to the base directory 'gpudrive'
working_dir = Path.cwd()
while working_dir.name != 'gpudrive':
    working_dir = working_dir.parent
    if working_dir == Path.home():
        raise FileNotFoundError("Base directory 'gpudrive' not found")
os.chdir(working_dir)

from gpudrive.env.config import EnvConfig
from gpudrive.env.env_torch import GPUDriveTorchEnv
from gpudrive.visualize.utils import img_from_fig
from gpudrive.env.dataset import SceneDataLoader

### Settings

In [2]:
MAX_NUM_OBJECTS = 64  # Maximum number of objects in the scene we control
NUM_WORLDS = 2  # Number of parallel environments
UNIQUE_SCENES = 2 # Number of unique scenes

device = 'cuda' # for simplicity purposes in notebook we use cpu, note that the simulator is optimized for GPU so use cuda if possible

### Initializing environments

- We provide both a torch and jax gymnasium interface with the simulator. Most functionality is specified in the `GPUDriveGymEnv` class in the `base_env`, `torch_env` and `jax_env` both inherit from the `GPUDriveGymEnv`, the only difference between these is that one exports torch tensors and the other jax arrays.
- All environment settings are defined in the `EnvConfig` dataclass. 


In [3]:
env_config = EnvConfig(
    steer_actions = torch.round(
        torch.linspace(-1.0, 1.0, 3), decimals=3),
    accel_actions = torch.round(
        torch.linspace(-3, 3, 3), decimals=3
    )
)

---

> 💡 **See the gym environment docs at [`pygpudrive/env`](https://github.com/Emerge-Lab/gpudrive/tree/main/pygpudrive/env)**

---

In [4]:
# Make dataloader
data_loader = SceneDataLoader(
    root="/workspace/data/gpu_drive/validation", # Path to the dataset
    batch_size=NUM_WORLDS, # Batch size, you want this to be equal to the number of worlds (envs) so that every world receives a different scene
    dataset_size=UNIQUE_SCENES, # Total number of different scenes we want to use
    sample_with_replacement=False, 
    seed=42, 
    shuffle=True,   
)

# Make environment
env = GPUDriveTorchEnv(
    config=env_config,
    data_loader=data_loader,
    max_cont_agents=MAX_NUM_OBJECTS, # Maximum number of agents to control per scenario
    device=device,
    smart_pkl_root="/workspace/data/smart"
)

Initialization finished
✓ Loaded 2 pickle files for current batch


In [5]:
env.data_batch

['/workspace/data/gpu_drive/validation/tfrecord-00000-of-00150_108.json',
 '/workspace/data/gpu_drive/validation/tfrecord-00000-of-00150_103.json']

In [6]:
env.get_scenario_ids()

{0: '1e41e4f90a5cae0a', 1: '7fcb65089a2bb672'}

In [8]:
env.smart_pkl_data

[{'map_save': {'traj_pos': tensor([[[2204.1599,  932.8286],
            [2203.1602,  932.8472],
            [2201.6626,  932.8751]],
   
           [[2177.2710,  919.2110],
            [2177.2800,  921.7110],
            [2177.2891,  924.2109]],
   
           [[2177.2891,  924.2109],
            [2177.2981,  926.7109],
            [2177.3071,  929.2109]],
   
           ...,
   
           [[2022.5867,  892.8409],
            [2024.9448,  892.0106],
            [2027.3029,  891.1804]],
   
           [[2027.3029,  891.1804],
            [2028.4366,  890.7812],
            [2026.0785,  891.6115]],
   
           [[2026.0785,  891.6115],
            [2024.6637,  892.1096],
            [2023.5299,  892.5088]]]),
   'traj_theta': tensor([ 3.1230,  1.5669,  1.5674,  ..., -0.3385, -0.3385,  2.8031])},
  'pt_token': {'type': tensor([1, 1, 1,  ..., 9, 9, 9], dtype=torch.uint8),
   'pl_type': tensor([0, 0, 0,  ..., 3, 3, 3], dtype=torch.uint8),
   'light_type': tensor([0, 0, 0,  ..., 0, 0, 0],

### Rollout 

A single rollout (one episode) is implemented as follows. We:
- step through N worlds simultaneously.
- render the simulator state using `plot_simulator_state()`

In [12]:
obs = env.reset()

frames = {f"env_{i}": [] for i in range(NUM_WORLDS)}

for t in range(env_config.episode_len):
    
    # Sample random actions
    rand_action = torch.Tensor(
        [[env.action_space.sample() for _ in range(MAX_NUM_OBJECTS * NUM_WORLDS)]]
    ).reshape(NUM_WORLDS, MAX_NUM_OBJECTS)

    # Step the environment
    env.step_dynamics(rand_action)

    obs = env.get_obs()
    reward = env.get_rewards()
    done = env.get_dones()

    # Render the environment    
    if t % 5 == 0:
        imgs = env.vis.plot_simulator_state(
            env_indices=list(range(NUM_WORLDS)),
            time_steps=[t]*NUM_WORLDS,
            zoom_radius=70,
        )
    
        for i in range(NUM_WORLDS):
            frames[f"env_{i}"].append(img_from_fig(imgs[i])) 
        
    if done.all():
        break

### Display videos of agents taking random actions

In [ ]:
mediapy.show_videos(frames, fps=5, width=500, height=500, columns=2, codec='gif')